In [8]:
import numpy as np
import pandas as pd

In [9]:
data = pd.read_csv("/content/HousingData.csv")
data.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


In [10]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [11]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val  
          
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    def r2_score(y_true, y_pred):
  
        # Calculate the mean of the true values
        mean_true = sum(y_true) / len(y_true)
        
        # Calculate the total sum of squares
        ss_tot = sum((y_true - mean_true)**2)
        
        # Calculate the residual sum of squares
        ss_res = sum((y_true - y_pred)**2)
        
        # Calculate the R-squared score
        r2 = 1 - (ss_res / ss_tot)
        

        return r2
    def mean_squared_error(y_true, y_pred):
   
      # Check if the lengths of both arrays are equal
      if len(y_true) != len(y_pred):
          raise ValueError("Length of y_true and y_pred should be the same.")
      
      # Calculate the squared differences between the true and predicted values
      squared_differences = [(y_true[i] - y_pred[i])**2 for i in range(len(y_true))]
      
      # Calculate the mean of the squared differences
      mse = sum(squared_differences) / len(squared_differences)
      
      return mse


In [12]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
#from sklearn.model_selection import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)
#print(Y)

In [13]:
import random

def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:
        random.seed(random_state)
    
    n = len(X)
    test_data = set(random.sample(range(n), int(n * test_size)))
    train_data = set(range(n)) - test_data
    
    X_train = [X[i] for i in train_data]
    X_test = [X[i] for i in test_data]
    y_train = [y[i] for i in train_data]
    y_test = [y[i] for i in test_data]
    return X_train, X_test, y_train, y_test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [14]:
regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=3)
regressor.fit(X_train,Y_train)
regressor.print_tree()

X_12 <= 7.79 ? 38.868566241464464
 left:X_5 <= 7.42 ? 35.947910609622014
  left:X_5 <= 6.635 ? 11.889229350159276
    left:X_0 <= 4.55587 ? 14.009386094674554
        left:24.662499999999998
        right:50.0
    right:X_0 <= 1.22358 ? 8.92397695562157
        left:32.023809523809526
        right:23.7
  right:X_0 <= 2.01019 ? 25.768664518438957
    left:X_10 <= 17.8 ? 7.156033702677746
        left:46.68666666666666
        right:40.125
    right:21.9
 right:X_12 <= 14.98 ? 11.41319204397367
  left:X_10 <= 13.0 ? 3.287611930735695
    left:X_5 <= 5.56 ? 16.646399999999996
        left:22.8
        right:33.0
    right:X_0 <= 14.3337 ? 1.5251336454012439
        left:21.0455223880597
        right:13.633333333333333
  right:X_0 <= 5.66637 ? 8.253625379195391
    left:X_6 <= 62.0 ? 2.7233520975056695
        left:21.766666666666666
        right:16.646938775510204
    right:X_12 <= 19.69 ? 2.799414097582366
        left:14.21578947368421
        right:10.635000000000002


In [15]:
y_pred = regressor.predict(X_test)

In [16]:
y_pred

[21.766666666666666,
 16.646938775510204,
 16.646938775510204,
 21.0455223880597,
 21.0455223880597,
 21.0455223880597,
 24.662499999999998,
 16.646938775510204,
 21.0455223880597,
 16.646938775510204,
 21.0455223880597,
 24.662499999999998,
 24.662499999999998,
 16.646938775510204,
 21.0455223880597,
 21.0455223880597,
 21.0455223880597,
 21.0455223880597,
 21.0455223880597,
 21.0455223880597,
 21.766666666666666,
 24.662499999999998,
 21.0455223880597,
 21.766666666666666,
 24.662499999999998,
 21.0455223880597,
 24.662499999999998,
 21.0455223880597,
 14.21578947368421,
 21.0455223880597,
 16.646938775510204,
 16.646938775510204,
 21.0455223880597,
 16.646938775510204,
 16.646938775510204,
 16.646938775510204,
 16.646938775510204,
 16.646938775510204,
 16.646938775510204,
 46.68666666666666,
 46.68666666666666,
 21.0455223880597,
 21.0455223880597,
 46.68666666666666,
 23.7,
 33.0,
 46.68666666666666,
 46.68666666666666,
 21.0455223880597,
 21.0455223880597,
 21.0455223880597,
 16.6

In [23]:
Y_pred = regressor.predict(X_test) 
np.sqrt(mean_squared_error(Y_test, Y_pred))

6.515375348927506